# Tests



In [1]:
import molsysmt as msm
from molsysmt import systems

In [2]:
molsys_1 = msm.convert(systems['alanine dipeptide']['alanine_dipeptide.h5msm'], to_form='molsysmt.MolSys')

In [3]:
molsys_2 = msm.structure.translate(molsys_1, translation='[0.5, 0.0, 0.0] nm')

In [4]:
molsys_merged = msm.merge([molsys_1, molsys_2], keep_ids=False)

In [5]:
msm.build.define_new_chain(molsys_merged)

In [6]:
molsys_merged.topology.chains

,chain_id,chain_name,chain_type
0,0,A,system


In [7]:
view = msm.view(molsys_merged, viewer='NGLView')

In [10]:
comparison = msm.compare(view, molsys_merged, attribute_type='topological',
             coordinates=True)

In [11]:
comparison

True

In [ ]:
    molsys_1 = msm.convert(systems['alanine dipeptide']['alanine_dipeptide.h5msm'], to_form='molsysmt.MolSys')
    molsys_2 = msm.structure.translate(molsys_1, translation='[0.5, 0.0, 0.0] nm')
    molsys_merged = msm.merge([molsys_1, molsys_2], keep_ids=False)
    msm.build.define_new_chain(molsys_merged, selection='all', chain_id=0, chain_name='A')
    view = msm.view(molsys_merged, viewer='NGLView')
    comparison = msm.compare(view, molsys_merged, attribute_type='topological',
            coordinates=True)
    assert comparison
